In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pathlib import Path
len(list((Path.home() / 'data/e-delib/kialo/kialoV2').glob('**/*.pkl')))

In [ ]:
len(list((Path.home() / 'data/e-delib/kialo/kialoV2/english').glob('**/*.pkl')))

In [ ]:
len(list((Path.home() / 'data/e-delib/kialo/kialoV2/other').glob('**/*.pkl')))

In [ ]:
import pandas as pd
from pathlib import Path

from kialo_util import read_data

maps = read_data({'local': True, 'debug_maps_size': 10, 'debug_maps_size': None, 'lang': None})

In [ ]:
maps[0].name

In [ ]:
import pandas as pd

maps_df = pd.DataFrame([{'id': m.id, 'size': m.number_of_children(), 'name': m.name} for m in maps]).set_index('id')
maps_df

## Analyse Map Results 

In [ ]:
from pathlib import Path
import json
def get_results_path(experiment):    
    data_path = Path.home() / 'data/delib-results/results'
    return data_path / experiment / '-results'

In [ ]:
# experiment = 'kialov2-no_hard_negatives-detailed'
experiment = 'use_templates-train_10000'
# experiment = 'no_templates-train_10000'

results_path = get_results_path(experiment)

# with open(results_path / 'all/all_maps.json') as data_faile:    
#     data = json.load(data_file)  
# df = pd.DataFrame.from_dict(data, orient='index')
# df = pd.json_normalize(df)
# df
    
df = pd.read_json(results_path / 'all/all_maps.json', orient='index')
# display(df)
# df.iloc[0]['all']['mrr']
df_dict = {k: pd.json_normalize(df[k].apply(lambda x: {} if pd.isna(x) else x)).set_index(df.index) for k in df.columns}
df_dict['only_leafs_limited_types']

In [ ]:
df_dict.keys()

In [ ]:
results_df = df_dict['only_leafs_limited_types'].join(maps_df)
results_df

In [ ]:
results_df[results_df['size']<=20]

In [ ]:
for s in [20, 50, 100, 500, 1000, 2000]:
    print(f"<{s:<10}{results_df[results_df['size']<s]['p5'].mean():.4f}")
print(f">={s:<10}{results_df[results_df['size']>=s]['p5'].mean():.4f}")

In [ ]:
results_df.hist('size');

In [ ]:
results_df.plot(x='size', y='p5', kind='scatter');

In [ ]:
import seaborn as sns
# sns.lmplot(x='size', y='p5', data=results_df)
# sns.kdeplot(y='size', x='p5', data=results_df)
# sns.kdeplot(results_df[['size', 'p5']])

## Analyse Node Results 

In [ ]:
import random
# experiment = 'kialov2-no_hard_negatives-detailed'
experiment = 'annotated_samples_in_test-baseline'
# experiment = 'use_templates-trained'
# experiment = 'use_templates-train_10000'
# experiment = 'no_templates-train_10000'

import json
all_nodes_results = json.loads((get_results_path(experiment) / 'all/all_nodes.json').read_text())
argument_maps_dict = {str(x.id):{n.id:n for n in x.all_children} for x in maps}

### double-check results

In [ ]:
errors=[]
for map_id, v in all_nodes_results.items():
    if v['only_leafs_limited_types']:
        for x in v['only_leafs_limited_types']:
            if x[1]==1 and x[2]!=argument_maps_dict[map_id][x[0]].parent.id:
                errors.append(f"c={x[0]} predicted={x[2]} actual={argument_maps_dict[map_id][x[0]].parent.id} {x}")
#                 print(errors[-1])
assert len(errors)==0, f'{len(errors)} {errors[0]}'

### show results 

In [ ]:
import random
node_id = '6492.1133'
random_node = True

map_id = int(node_id.split('.')[0])
argument_map_results = all_nodes_results[str(map_id)]['only_leafs_limited_types']
argument_map_results = {x[0]: x for x in argument_map_results}
argument_map = next((x for x in maps if x.id == map_id))

In [ ]:
if random_node:
    node_id = random.choice(list(argument_map_results))
    
node_results = argument_map_results[node_id]
node = next((x for x in argument_map.all_children if x.id == node_id))
predicted_node = next((x for x in argument_map.all_children if x.id == node_results[2]))
print(f"{node_results[0]} parent {node_results[2]} rank={node_results[1]} dist={node_results[3]} vs \n{node.id} parent {node.parent.id}")
print(f"child:\n{node.name}")
print(f"actual:\n{node.parent.name}")
print(f"predicted:\n{predicted_node.name}")